# 1. The Thompson Sampling Algorithm

Thompson sampling is an algorithm for online decision problems where actions are taken sequentially in a manner that must balance between exploiting what is known to maximize immediate performance and investing to accumulate new information that may improve future performance. The colorful name for our problem comes from a motivating story in which a gambler enters a casino and sits down at a slot machine with multiple levers, or arms, that can be pulled. When pulled, an arm produces a random payout drawn independently of the past. Because the distribution of payouts corresponding to each arm is not listed, the player can learn it only by experimenting. As the gambler learns about the arms’ payouts, she faces a dilemma: in the immediate future she expects to earn more by exploiting arms that yielded high payouts in the past, but by continuing to explore alternative arms she may learn how to earn higher payouts in the future. Can she develop a sequential strategy for pulling arms that balances this tradeoff and maximizes the cumulative payout earned?

Suppose the agent applies a sequence of actions $x_1,x_2,x_3,...$ to a system, selecting each from a set $X$. After applying action $x_t$, the agent observes an outcome $y_t$, which the system randomly generates according to a conditional probability measure $q_θ(·|x_t)$. The agent enjoys a reward $r_t = r(y_t)$, where $r$ is a known function. The agent is initially uncertain about the value of $θ$ and represents his uncertainty using a prior distribution $p$.

**Algorithm** - $\text{Thompson}(X,p,q,r)$ <br>
1: **for** $t = 1,2,...$ **do** <br> 
2: $\space$ $\space$ # sample model: <br>
3: $\space$ $\space$ Sample $\hat{θ} ∼ p$ <br>
4: <br>
5: $\space$ $\space$ #select and apply action: <br>
6: $\space$ $\space$ $x_t ← \text{argmax}_{x \in X} E_{q_θ}[r(y_t)|x_t = x]$ <br>
7: $\space$ $\space$ Apply $x_t$ and observe $y_t$ <br>
8: $\space$ $\space$ <br>
9: $\space$ $\space$ # update distribution: <br>
10: $\space$ $\space$ $p ← P_{p,q}(θ \in ·| x_t, y_t)$ <br>
11: **end for**

The Bernoulli bandit with a beta prior serves as a special case of this more general formulation. In this special case, the set of actions is $X ={1,...,K}$ and only rewards are observed, so $y_t = r_t$. Observations and rewards are modeled by conditional probabilities $q_θ(1|k) = θ_k$ and $q_θ(0|k) = 1 −θ_k$. 

The prior distribution is encoded by vectors $α$ and $β$, with probability density function given by:

$$
p_\theta = \frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha_k) \Gamma(\beta_k)} \theta_{k}^{a_k - 1} (1 - \theta_k)^{\beta_k - 1}
$$

where $\Gamma$ denotes the gamma function. In other words, under the prior distribution, components of $θ$ are independent and beta-distributed, with parameters $α$ and $β$.

For this problem, the TS begins each $t$-th iteration with posterior parameters $(α_k,β_k)$ for $k ∈ {1,...,K}$. It randomly draws $\hat{θ}_k$ from a beta distribution with parameters $(α_k,β_k)$ and then selects the action $x$ that maximizes $E_{q_θ} [r(y_t)|x_t = x] = \hat{θ}_k$. After applying the selected action, a reward $r_t = y_t$ is observed, and belief distribution parameters are updated according to:
$$
(α,β) ← (α + r_t 1_{x_t} ,β + (1 − r_t)1_{x_t} )
$$

where $1_{x_t}$ is a vector with component $x_t$ equal to $1$ and all other components equal to $0$.

# 2. Modelling Considerations

Our discussions above has centered around a somewhat idealised view of TS, which ignored the process of prior specification and assumed a simple model in which the system and set of feasible actions is constant over time and there is no side information on decision context.

Therefore, in the following sections we will focus on the decision of the prior distribution

## 2.1. Prior Distribution Specification

The TS algorithm we have presented require as input a **prior distribution** over model parameters. The choice of prior can be important so let's think of a banner ad placement problem.

There are $K$ banner ads for a single product, with unknown click-through probabilities $(θ_1, . . . ,θ_K)$. Given a prior, TS can learn to select the most successful ad. We could use a uniform or, equivalently, a $Beta(1,1)$ distribution over each $θ_k$. However, if some values of θk are more likely than others, using a uniform prior sacrifices performance. In particular, this prior represents no understanding of the context, ignoring any useful knowledge from past experience. Taking knowledge into account reduces what must be learned and therefore reduces the time it takes for TS to identify the most effective ads.